# Online Retail Sales Analysis

End-to-end exploratory analysis for the transformed Online Retail II dataset. Run the ETL scripts (`src/elt.py`, `src/load.py`, `src/transform.py`) before executing this notebook to ensure `data/retail.sqlite` is populated.


## Notebook Roadmap
- [Setup](#Setup)
- [Data Snapshot](#Data-Snapshot)
- [Monthly Revenue Trend](#Monthly-Revenue-Trend)
- [Top Products](#Top-Products)
- [Top Customers](#Top-Customers)
- [Revenue by Region & Country](#Revenue-by-Region-\&-Country)
- [Invoice Basket Analysis](#Invoice-Basket-Analysis)
- [Customer Breadth & Repeat Behaviour](#Customer-Breadth-\&-Repeat-Behaviour)
- [Seasonal & Product Insights](#Seasonal-\&-Product-Insights)
- [Cleanup](#Cleanup)


## Setup
Import libraries, connect to SQLite, and define a helper for executing SQL queries.


In [ ]:
from pathlib import Path
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

plt.style.use('seaborn-v0_8')

PROJECT_ROOT = Path().resolve().parent
DB_PATH = PROJECT_ROOT / 'data' / 'retail.sqlite'

if not DB_PATH.exists():
    raise FileNotFoundError(f'Database not found at {DB_PATH}. Run the ETL pipeline before executing the notebook.')

conn = sqlite3.connect(DB_PATH)

pd.set_option('display.float_format', lambda value: f"{value:,.2f}")

def run_sql(query: str, params=None) -> pd.DataFrame:
    """Execute a SQL query against the warehouse and return a DataFrame."""
    print(query.strip())
    return pd.read_sql_query(query, conn, params=params)


## Data Snapshot
Check table sizes and preview fact data to validate the warehouse load.


In [ ]:
table_counts_sql = """
SELECT 'dim_customer' AS table_name, COUNT(*) AS row_count FROM dim_customer
UNION ALL
SELECT 'dim_product', COUNT(*) FROM dim_product
UNION ALL
SELECT 'fact_sales', COUNT(*) FROM fact_sales
UNION ALL
SELECT 'stg_sales', COUNT(*) FROM stg_sales;
"""
table_counts = run_sql(table_counts_sql)
display(table_counts)


In [ ]:
sample_fact_sql = """
SELECT invoice_no, invoice_date, stock_code, customer_id, quantity, unit_price, revenue
FROM fact_sales
ORDER BY invoice_date
LIMIT 5;
"""
sample_fact = run_sql(sample_fact_sql)
display(sample_fact)


## Monthly Revenue Trend
Understand revenue trajectory, order volumes, and invoice values over time.


In [ ]:
monthly_revenue_sql = """
SELECT strftime('%Y-%m', invoice_date) AS month,
       ROUND(SUM(revenue), 2) AS monthly_revenue,
       COUNT(DISTINCT invoice_no) AS invoices,
       COUNT(*) AS order_lines,
       ROUND(SUM(revenue) / NULLIF(COUNT(DISTINCT invoice_no), 0), 2) AS avg_invoice_value
FROM fact_sales
GROUP BY month
ORDER BY month;
"""
monthly_revenue = run_sql(monthly_revenue_sql)
display(monthly_revenue.tail())


In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
ax[0].plot(pd.to_datetime(monthly_revenue['month']), monthly_revenue['monthly_revenue'], marker='o', color='#1f77b4')
ax[0].set_title('Monthly Revenue')
ax[0].set_ylabel('Revenue (GBP)')
ax[0].grid(True, alpha=0.3)

ax[1].bar(pd.to_datetime(monthly_revenue['month']), monthly_revenue['avg_invoice_value'], color='#ff7f0e')
ax[1].set_title('Average Invoice Value')
ax[1].set_ylabel('Revenue per Invoice (GBP)')
ax[1].grid(True, axis='y', alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


> **Takeaway:** Revenue peaks during Q4 2010, confirming the holiday uplift. Average invoice values climb steadily throughout 2010, indicating larger order sizes.


## Top Products
Identify the SKUs generating the highest revenue and volume.


In [ ]:
top_products_sql = """
SELECT p.stock_code,
       p.description,
       SUM(f.quantity) AS total_quantity,
       ROUND(SUM(f.revenue), 2) AS total_revenue
FROM fact_sales f
JOIN dim_product p
  ON f.stock_code = p.stock_code
GROUP BY p.stock_code, p.description
ORDER BY total_revenue DESC
LIMIT 15;
"""
top_products = run_sql(top_products_sql)
display(top_products)


> **Observation:** Giftware and lighting SKUs dominate revenue. Normalise variant descriptions (e.g., `RETROSPOT` vs `RETRO SPOT`) to consolidate reporting.


## Top Customers
Highlight the most valuable customers and their regions.


In [ ]:
top_customers_sql = """
SELECT f.customer_id,
       c.country,
       c.region,
       COUNT(DISTINCT f.invoice_no) AS invoices,
       ROUND(SUM(f.revenue), 2) AS total_revenue,
       ROUND(AVG(f.revenue), 2) AS avg_line_revenue
FROM fact_sales f
JOIN dim_customer c
  ON f.customer_id = c.customer_id
GROUP BY f.customer_id, c.country, c.region
ORDER BY total_revenue DESC
LIMIT 15;
"""
top_customers = run_sql(top_customers_sql)
display(top_customers)


> **Observation:** A handful of wholesale customers in the UK and Netherlands contribute a large share of revenue. Focus retention efforts on these accounts.


## Revenue by Region & Country
Break down performance by geographic segment.


In [ ]:
region_revenue_sql = """
SELECT c.region,
       ROUND(SUM(f.revenue), 2) AS total_revenue,
       COUNT(DISTINCT c.customer_id) AS customers,
       ROUND(AVG(f.revenue), 2) AS avg_line_revenue,
       ROUND(100.0 * SUM(f.revenue) / SUM(SUM(f.revenue)) OVER (), 2) AS revenue_share_pct
FROM fact_sales f
JOIN dim_customer c
  ON f.customer_id = c.customer_id
GROUP BY c.region
ORDER BY total_revenue DESC;
"""
region_revenue = run_sql(region_revenue_sql)
display(region_revenue)


In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
sns.barplot(data=region_revenue, x='region', y='total_revenue', palette='Blues_d', ax=ax)
ax.set_title('Revenue by Region')
ax.set_ylabel('Total Revenue (GBP)')
ax.set_xlabel('Region')
plt.tight_layout()
plt.show()


In [ ]:
country_revenue_sql = """
SELECT c.country,
       COUNT(DISTINCT f.invoice_no) AS invoices,
       ROUND(SUM(f.revenue), 2) AS total_revenue,
       ROUND(SUM(f.revenue) / NULLIF(COUNT(DISTINCT f.invoice_no), 0), 2) AS avg_invoice_value
FROM fact_sales f
JOIN dim_customer c
  ON f.customer_id = c.customer_id
GROUP BY c.country
HAVING COUNT(DISTINCT f.invoice_no) >= 20
ORDER BY total_revenue DESC
LIMIT 15;
"""
country_revenue = run_sql(country_revenue_sql)
display(country_revenue)


> **Takeaway:** The UK leads in total revenue, but export markets such as the Netherlands and Australia show higher basket sizes, suggesting wholesale strength abroad.


## Invoice Basket Analysis
Review high-value orders and revenue contribution by invoice size.


In [ ]:
top_invoices_sql = """
SELECT invoice_no,
       MIN(invoice_date) AS invoice_date,
       COUNT(*) AS line_items,
       ROUND(SUM(revenue), 2) AS total_revenue,
       ROUND(SUM(quantity), 0) AS total_units
FROM fact_sales
GROUP BY invoice_no
ORDER BY total_revenue DESC
LIMIT 15;
"""
top_invoices = run_sql(top_invoices_sql)
display(top_invoices)


In [ ]:
basket_distribution_sql = """
WITH invoice_stats AS (
    SELECT invoice_no,
           SUM(revenue) AS total_revenue,
           SUM(quantity) AS total_units
    FROM fact_sales
    GROUP BY invoice_no
)
SELECT
    CASE
        WHEN total_revenue < 100 THEN '< £100'
        WHEN total_revenue < 250 THEN '£100-£249'
        WHEN total_revenue < 500 THEN '£250-£499'
        WHEN total_revenue < 1000 THEN '£500-£999'
        ELSE '£1K+'
    END AS revenue_band,
    COUNT(*) AS invoices,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS share_pct
FROM invoice_stats
GROUP BY revenue_band
ORDER BY invoices DESC;
"""
basket_distribution = run_sql(basket_distribution_sql)
display(basket_distribution)


> **Insight:** Large invoices (£500+) represent a small fraction of orders but drive a significant revenue share, warranting premium customer engagement.


## Customer Breadth & Repeat Behaviour
Measure how broadly customers purchase across the catalogue and their repeat tendencies.


In [ ]:
product_breadth_sql = """
SELECT customer_id,
       COUNT(DISTINCT stock_code) AS distinct_products,
       COUNT(DISTINCT invoice_no) AS invoices,
       ROUND(SUM(revenue), 2) AS total_revenue
FROM fact_sales
GROUP BY customer_id
HAVING invoices >= 5
ORDER BY distinct_products DESC
LIMIT 15;
"""
product_breadth = run_sql(product_breadth_sql)
display(product_breadth)


In [ ]:
repeat_customers_sql = """
WITH customer_orders AS (
    SELECT customer_id,
           COUNT(DISTINCT invoice_no) AS invoices,
           SUM(revenue) AS total_revenue
    FROM fact_sales
    GROUP BY customer_id
)
SELECT
    SUM(CASE WHEN invoices = 1 THEN 1 ELSE 0 END) AS single_purchase_customers,
    SUM(CASE WHEN invoices > 1 THEN 1 ELSE 0 END) AS repeat_customers,
    ROUND(100.0 * SUM(CASE WHEN invoices > 1 THEN 1 ELSE 0 END) / COUNT(*), 2) AS repeat_customer_rate_pct,
    ROUND(AVG(total_revenue), 2) AS avg_customer_revenue
FROM customer_orders;
"""
repeat_customers = run_sql(repeat_customers_sql)
display(repeat_customers)


> **Observation:** Repeat buyers make up a majority of the customer base and yield higher lifetime revenue — retention programs should be prioritised.


## Seasonal & Product Insights
Investigate seasonal SKUs and category dynamics.


In [ ]:
seasonal_products_sql = """
SELECT p.description,
       SUM(f.quantity) AS total_units,
       ROUND(SUM(f.revenue), 2) AS total_revenue
FROM fact_sales f
JOIN dim_product p
  ON f.stock_code = p.stock_code
WHERE UPPER(p.description) LIKE '%CHRISTMAS%'
GROUP BY p.description
ORDER BY total_units DESC
LIMIT 10;
"""
seasonal_products = run_sql(seasonal_products_sql)
display(seasonal_products)


In [ ]:
monthly_category_sql = """
WITH monthly_product AS (
    SELECT strftime('%Y-%m', invoice_date) AS month,
           CASE
               WHEN UPPER(p.description) LIKE '%LIGHT%' THEN 'Lighting'
               WHEN UPPER(p.description) LIKE '%MUG%' THEN 'Mugs'
               WHEN UPPER(p.description) LIKE '%BAG%' THEN 'Bags'
               WHEN UPPER(p.description) LIKE '%CARD%' THEN 'Cards'
               ELSE 'Other'
           END AS category,
           SUM(f.revenue) AS revenue
    FROM fact_sales f
    JOIN dim_product p
      ON f.stock_code = p.stock_code
    GROUP BY month, category
)
SELECT month,
       category,
       ROUND(revenue, 2) AS revenue
FROM monthly_product
WHERE month BETWEEN '2010-01' AND '2010-12'
ORDER BY month, revenue DESC;
"""
monthly_category = run_sql(monthly_category_sql)
display(monthly_category.head(15))


> **Insight:** Holiday-themed inventory spikes sharply in Q4, especially Christmas decorations and lighting. Everyday categories like bags and mugs provide steady revenue across the year.


## Cleanup
Close the SQLite connection once analysis is complete.


In [ ]:
conn.close()
